In [ ]:
## Import libraries

import pandas as pd
import numpy as np
import xarray as xr
from glob import glob
from pathlib import Path
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.special import gamma
from scipy.interpolate import interp1d

In [ ]:
## Settings for calculation functions

# This is to get the version number of the calc_funcs script being used so that it
# can be appended to the file name of any outputs. The reason this is done is because
# the calculation functions below and the plot functions in the plot_funcs script both
# output intermediate files one at a time from low level to high level, and that each
# file name is used in recognising whether there is a need to re-run a script (if the
# file already exists then the script is not run so as to save on computation).
# However, this method can propagate errors from low level through to high level
# if there has been a change to the code and/or output at the lower levels. By
# appending the version number of the calc_funcs script being used, it forces all
# intermediate files to be recreated from scratch rather than reuse intermediate files
# which was outputted by outdated code. "v00" is used as a placeholder version number
# if there is an error: it is used mostly for scripting purposes within an
# interactive python notebook where the file name cannot be directly extracted
# using the __file__ python variable.
try:
    calc_funcs_ver = Path(__file__).stem[-3:]
except:
    calc_funcs_ver = "v00"

# Valid regions and their mapping to axis extents in [W, E, S, N] format 
# as well as timezones in hour +- GMT
regions = {
    # Central America (mostly Honduras-Nicaragua-Costa Rica)
    "ca": {"extent": [-91, -81, 7, 17], "tz": -6},
    # South America (mostly central and eastern Brazil)
    "sa": {"extent": [-65, -30, -15, 0], "tz": -3},
    # Western Australia (mostly near the west coast)
    "wa": {"extent": [113, 123, -35, -30], "tz": +8}
}

# Earliest and latest entries in each GLASS dataset
avhrr_earliest = "Jan-1981"
modis_earliest = "Mar-2000"
avhrr_latest = "Dec-2018"
modis_latest = "Dec-2021"
fapar_earliest = "Jan-1982"
fapar_latest = "Dec-2020"

# Size of chunks
chunksize = "500MB"

# Valid subsets to use as argument in climatologies and
# their mapping to month numbers for use in xarray time slicing
subsets = {
    "all": [1,2,3,4,5,6,7,8,9,10,11,12],
    "DJF": [12,1,2], "MAM": [3,4,5], "JJA": [6,7,8], "SON": [9,10,11],
    "Jan": [1], "Feb": [2], "Mar": [3], "Apr": [4], "May": [5], "Jun": [6],
    "Jul": [7], "Aug": [8], "Sep": [9], "Oct": [10], "Nov": [11], "Dec": [12]
}

# Valid variables for use in analysis
hours = list(range(0, 24))
vars_glass = ["lai", "fapar"]
vars_era5 = {
    "vars": {
        "surface": ["wv10", "wv100", "mslp", "t2", "slhf", "sshf"],
        "atmos": ["viec", "vipile", "vike", "tcclw", "tcwv", "nac"]
    },
    "dvars": {
        "surface": ["dwv10", "dwv100", "dmslp", "dt2", "dslhf", "dsshf"],
        "atmos": ["dviec", "dvipile", "dvike", "dtcclw", "dtcwv", "dnac"]
    }
}
vars_and_dvars_era5_all = (
    vars_era5["vars"]["surface"] + vars_era5["vars"]["atmos"] +
    vars_era5["dvars"]["surface"] + vars_era5["dvars"]["atmos"]
)

# File number check to make sure data_download notebook was run correctly
number_of_glass_files = {"lai": {"avhrr": 1748, "modis": 1005},
                         "fapar": {"avhrr": 1702, "modis": 960}
                        }
number_of_era5_hour_files = 42

# Resolution of ERA5 dataset in degrees (used for regridding)
res_era5 = 0.25

# Speed (in m/s) for expected rate of exceedance analysis at 100 m
speed_eroe = 42.5

# Typical power curve for a 100 m turbine with 100 m rotor diameter and 
# a nameplate rating of around 2500 kW (used to compute gross capacity factor)
# Speeds are in m/s, powers are in kW, data from https://www.thewindpower.net
speeds_common = np.append(np.linspace(0, 25.5, 52), 999)
power_nameplate = 2500
# Vestas V100/2600
powers_vestas = np.array([0, 0, 0, 0, 0, 0, 21, 63, 115, 172, 239, 318, 405, 550,
                          706, 890, 1080, 1283, 1485, 1641, 1796, 1944, 2092, 2225,
                          2351, 2440, 2502, 2560, 2584, 2597, 2600, 2600, 2600,
                          2600, 2600, 2600, 2600, 2600, 2600, 2600, 2600, 2600,
                          2600, 2600, 2600, 2600, 2600, 2600, 2600, 2600, 2600, 0, 0])
# Goldwind GW100/2500
powers_gw = np.array([0, 0, 0, 0, 0, 6, 34, 65, 101, 165, 235, 320, 409, 530, 655,
                      826, 997, 1196, 1394, 1669, 1943, 2170, 2313, 2415, 2458,
                      2485, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500,
                      2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500,
                      2500, 2500, 2500, 2500, 2500, 2500, 0, 0])
# GE Energy 2.5-100
powers_ge = np.array([0, 0, 0, 0, 0, 0, 10, 80, 160, 250, 340, 460, 590, 770, 952,
                      1170, 1389, 1650, 1869, 2100, 2260, 2400, 2487, 2500, 2500,
                      2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500,
                      2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500,
                      2500, 2500, 2500, 2500, 2500, 2500, 0, 0])
# Average
powers_avg = (powers_vestas/26*25 + powers_gw + powers_ge) / 3

# Names of calculation functions
calc_func_names = ["calc_glass_mean_climatology",
                   "calc_era5_mdp_climatology_given_var_or_dvar",
                   "calc_era5_mdp_climatology_stats_given_var_or_dvar",
                   "calc_era5_mdp_climatology_values_given_var_or_dvar_and_hour",
                   "calc_era5_weibull_climatology"]

In [ ]:
## Supplementary functions for calculation functions

def check_args(calc_func=None, region=None, period_start=None, period_end=None,
               subset=None, var_or_dvar=None, hour=None,
               vars_level=None, vars_type=None):
    
    """
    Function to check whether input arguments are valid.
    
    Arguments:
        calc_func (function): Calculation function to compute difference in
            results from. Must be one of: ["calc_glass_mean_climatology",
            "calc_era5_mdp_climatology_given_var_or_dvar",
            "calc_era5_mdp_climatology_stats_given_var_or_dvar",
            "calc_era5_mdp_climatology_values_given_var_or_dvar_and_hour",
            "calc_era5_weibull_climatology"].
        region (str): Region to perform calculation over.
            Must be one of: ["ca", "sa", "wa"].
        period1_start (datetime.datetime): Start of first period to perform
            calculation over. Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
            Note that this is a datetime.dateime object as opposed to str.
        period1_end (datetime.datetime): End of first period to perform
            calculation over. Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
            Note that this is a datetime.dateime object as opposed to str.
        period2_start (datetime.datetime): Start of second period to perform
            calculation over. Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
            Note that this is a datetime.dateime object as opposed to str.
        period2_end (datetime.datetime): End of second period to perform
            calculation over. Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
            Note that this is a datetime.dateime object as opposed to str.
        subset (str): Subset of period to perform calculation over.
            Must be one of: ["all", "DJF", "MAM", "JJA", "SON",
            "Jan", "Feb", "Mar", "Apr", "May", "Jun",
            "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"].
        var_or_dvar (str): Variable or value of change in variable to perform
            calculation over. Must be one of: ['wv10', 'wv100', 'mslp', 't2',
            'slhf', 'sshf', 'viec', 'vipile', 'vike', 'tcclw', 'tcwv', 'nac',
            'dwv10', 'dwv100', 'dmslp', 'dt2', 'dslhf', 'dsshf', 'dviec',
            'dvipile', 'dvike', 'dtcclw', 'dtcwv', 'dnac'].
        hour (int): Hour of mean diurnal profile to compute value for.
            Must be one of: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
            13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23].
        vars_level (str): Level from which to draw variables for analysis.
            This is used for the plot_func script. Must be one of:
            ["surface", "atmos"].
        vars_type (str): Whether to analyse the variables themselves or the change
            in their mean diurnal profile values as compared with their values
            in the previous hour. This is used for the plot_func script.
            Must be one of: ["vars", "dvars"].            
    
    Returns:
        AssertionError if any of the input arguments are invalid.
    """
    
    if calc_func:
        assert calc_func.__name__ in calc_func_names, \
            f"calc_func must be one of: {calc_func_names}"    
    if region:
        assert region in [*regions], \
            f"region must be one of: {[*regions]}"
    if period_start:
        assert period_start >= datetime.strptime(avhrr_earliest, "%b-%Y"), \
            f"period_start must be equal to or later than {avhrr_earliest}"        
    if period_end:
        assert period_end <= datetime.strptime(modis_latest, "%b-%Y"), \
            f"period_end must be equal to or earlier than {modis_latest}"
    if (period_start is not None) & (period_end is not None):
        assert period_end >= period_start, \
            "period_end must be equal to or later than period_start"        
    if subset:
        assert subset in [*subsets], \
            f"subset must be one of: {[*subsets]}"
    if (period_start is not None) & (period_end is not None) & (subset is not None):
        dates_in_period = pd.date_range(period_start, period_end, freq = "MS")
        months_in_period = set(map(int, dates_in_period.strftime("%-m")))
        assert any(month in subsets[subset] for month in months_in_period), \
            "period(s) must contain at least one month within the given subset"
    if var_or_dvar:
        assert var_or_dvar in vars_and_dvars_era5_all, \
            f"var_or_dvar must be one of: {vars_and_dvars_era5_all}"
    if hour:
        assert hour in hours, \
            f"hour must be one of: {hours}"
    if vars_level:
        assert vars_level in [*[*vars_era5.values()][0].keys()], \
            f"vars_level must be one of: {[*[*vars_era5.values()][0].keys()]}"
    if vars_type:
        assert vars_type in [*vars_era5], \
            f"vars_type must be one of: {[*vars_era5]}"

In [ ]:
def glass_data_source_to_use(period_start, period_end):
    
    """
    Select which GLASS data source (AVHRR or MODIS) to use.
    
    Arguments:
        period_start (datetime.datetime): Start of period to perform calculation over.
            Note that this is a datetime.dateime object as opposed to str.
        period_end (datetime.datetime): End of period to perform calculation over.
            Note that this is a datetime.dateime object as opposed to str.
        
    Returns:
        glass_data_source (str): String indicating whether to use
            "avhrr" or "modis" data for the given period.
    
    For the given period, select the most appropriate GLASS data source to use 
    (out of AVHRR and MODIS). MODIS is preferentially selected where the given 
    period is completely contained within the time range of MODIS data. 
    Otherwise, AVHRR data is used. Periods which simultaneously cover both an 
    AVHRR-only period (i.e. before Mar-2000) and a MODIS-only period 
    (i.e. after Dec-2018) are prevented from selection since summary statistics
    over this range is subject to artefacts from the change in instruments.
    """
    
    check_args(period_start=period_start, period_end=period_end)
    
    if ((period_start >= datetime.strptime(avhrr_earliest, "%b-%Y")) & 
        (period_start < datetime.strptime(modis_earliest, "%b-%Y")) &
        (period_end <= datetime.strptime(avhrr_latest, "%b-%Y"))
       ):
        glass_data_source = "avhrr"
    elif ((period_start >= datetime.strptime(modis_earliest, "%b-%Y")) &
          (period_end <= datetime.strptime(modis_latest, "%b-%Y"))
         ):
        glass_data_source = "modis"
    else:
        raise Exception("If period_start is before Mar-2000, " +
                        "period_end cannot be after Dec-2018 " +
                        "(since this would cover both an " +
                        "AVHRR-only and a MODIS-only period)")
    return glass_data_source

In [ ]:
def regrid_era5(ds):
    
    """
    Regrid ERA5 xarray dataset.
    
    Arguments:
        ds (xarray.Dataset): Dataset containing ERA5 data, loaded in
            with xarray using the netcdf4 engine.
                                
    Returns:
        ds (xarray.Dataset): Dataset with regridded coordinates.
        
    Shifts each latitude coordinate south and longitude coordinate east by
    half a grid cell. This reflects the fact that coordinates in the
    original ERA5 dataset defines the north-western corner of each grid
    cell, whereas xarray plots assuming the coordinates refer to the centre.
    """
    
    ds = (ds
          .assign_coords({"latitude": ds.latitude - res_era5/2,
                          "longitude": (ds.longitude + 180 + res_era5/2) % 360 - 180})
          # Redundant measure just in case longitudes exceed 180 degrees
          .sortby("longitude")
         )
    return ds

In [ ]:
def magnitude(x, y):
    
    """
    Calculate magnitude of 2D vectors given their components.
    
    Arguments:
        x (xarray.DataArray): x-component of vectors.
        y (xarray.DataArray): y-component of vectors.
        
    Returns:
        da (xarray.DataArray); Magnitude of vectors.
        
    Performs a vectorised computation on two different data arrays
    containing the x and y component of some vectors and returns
    the magnitude of the vectors. Dask is allowed.
    """
    
    func = lambda a, b: np.sqrt(a**2 + b**2)
    da = xr.apply_ufunc(func, x, y, dask = "allowed")
    return da

In [ ]:
def weibull_params(da_mean, da_std):
    
    """
    Obtain Weibull parameters for wind speed distribution from
    mean and standard deviation.
    
    Arguments:
        da_mean (xarray.DataArray): Mean wind speed.
        da_std (xarray.DataArray): Standard deviation of wind speed.
    
    Returns:
        da_c (xarray.DataArray): Scale parameter for empirical Weibull fit.
        da_k (xarray.DataArray): Shape parameter for empirical Weibull fit.
    
    Performs a vectorised computation on two different data arrays
    containing the mean and standard deviation of wind speed and returns
    the Weibull scale and shape parameters for the fit. Dask is allowed.
    This method uses equations (15) and (16) from an article by Justus et al.
    (1977) titled "Methods for Estimating Wind Speed Frequency Distributions".
    """
    
    k = lambda mean, std: (std / mean)**(-1.086)
    da_k = xr.apply_ufunc(k, da_mean, da_std, dask = "allowed")
    c = lambda mean, k: mean / gamma(1 + 1/k)
    da_c = xr.apply_ufunc(c, da_mean, da_k, dask = "allowed")
    return da_c, da_k

In [ ]:
def weibull_eroe(da_c, da_k, ws):
    
    """
    Obtain the expected rate of exceedance for a particular wind speed
    from fitted Weibull parameters.
    
    Arguments:
        da_c (xarray.DataArray): Scale parameter for empirical Weibull fit.
        da_k (xarray.DataArray): Shape parameter for empirical Weibull fit.
        ws (float or int): Particular wind speed on which to conduct the
            expected rate of exceedance analysis.
        
    Returns:
        da_eroe (xarray.DataArray): Expected rate of exceedance from Weibull.
        
    For the given Weibull parameters, the expected rate of exceedance for a 
    particular wind speed is computed as 1 minus the cumulative probability
    distribution for the Weibull fit.
    """
    
    assert (isinstance(ws, float) | isinstance(ws, int)), \
        "ws must have data type float or int"
    
    eroe = lambda c, k: np.exp(-(ws / c)**k)
    da_eroe = xr.apply_ufunc(eroe, da_c, da_k, dask = "allowed")
    return da_eroe

In [ ]:
def gcf(da_ws, speeds, powers, power_max):
    
    """
    Compute the gross capacity factor for a typical wind turbine.
    
    Arguments:
        da_ws (xarray.DataArray): Wind speed data over a period.
        speeds (numpy.ndarray): Speed bins for the turbine's power curve.
        powers (numpy.ndarray): Powers for each speed bin according to manufacturer
            power curve data.
        power_max (float or int): The maximum power which the turbine can produce.
        
    Returns:
        da_gcf (xarray.DataArray): Gross capacity factor over the period.
        
    First uses the speeds and powers arguments to produce an interpolation
    function for the power curve. Then this interpolation function is applied
    to obtain the power at each wind speed data point in da_ws. Finally, this
    is divided over the maximum power and averaged over the period to obtain
    the gross capacity factor.
    """
    
    assert speeds.ndim == 1, \
        "speeds must be a 1D numpy array with data type 'float64' or 'int64'"
    assert (speeds.dtype == "float64") | (speeds.dtype == "int64"), \
        "speeds must be a 1D numpy array with data type 'float64' or 'int64'"
    assert powers.ndim == 1, \
        "powers must be a 1D numpy array with data type 'float64' or 'int64'"
    assert (powers.dtype == "float64") | (powers.dtype == "int64"), \
        "powers must be a 1D numpy array with data type 'float64' or 'int64'"
    assert isinstance(power_max, float) | isinstance(power_max, int), \
        "power_max must have data type 'float' or 'int'"
    
    power_curve = interp1d(speeds, powers, kind = "nearest")
    gcf_instant = lambda ws: power_curve(ws) / power_max
    da_gcf = (xr.apply_ufunc(gcf_instant, da_ws, dask = "allowed")
              .mean("time")
             )
    return da_gcf

In [ ]:
def get_paths_for_calc_diff(calc_func, region, period1_start, period1_end, period2_start,
                            period2_end, subset, var_or_dvar=None, hour=None):
    
    """
    Obtain output path for calc_diff function, as well as paths for the calc_func
    outputs from each given period.
    
    Arguments:
        calc_func (function): Calculation function to compute difference in
            results from. Must be one of: ["calc_glass_mean_climatology",
            "calc_era5_mdp_climatology_given_var_or_dvar",
            "calc_era5_mdp_climatology_stats_given_var_or_dvar",
            "calc_era5_mdp_climatology_values_given_var_or_dvar_and_hour",
            "calc_era5_weibull_climatology"].
        region (str): Region to perform calculation over.
            Must be one of: ["ca", "sa", "wa"].
        period1_start (datetime.datetime): Start of first period to perform
            calculation over. Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
            Note that this is a datetime.dateime object as opposed to str.
        period1_end (datetime.datetime): End of first period to perform
            calculation over. Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
            Note that this is a datetime.dateime object as opposed to str.
        period2_start (datetime.datetime): Start of second period to perform
            calculation over. Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
            Note that this is a datetime.dateime object as opposed to str.
        period2_end (datetime.datetime): End of second period to perform
            calculation over. Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
            Note that this is a datetime.dateime object as opposed to str.
        subset (str): Subset of period to perform calculation over.
            Must be one of: ["all", "DJF", "MAM", "JJA", "SON",
            "Jan", "Feb", "Mar", "Apr", "May", "Jun",
            "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"].
        var_or_dvar (str): Variable or value of change in variable to perform
            calculation over. Must be one of: ['wv10', 'wv100', 'mslp', 't2',
            'slhf', 'sshf', 'viec', 'vipile', 'vike', 'tcclw', 'tcwv', 'nac',
            'dwv10', 'dwv100', 'dmslp', 'dt2', 'dslhf', 'dsshf', 'dviec',
            'dvipile', 'dvike', 'dtcclw', 'dtcwv', 'dnac'].
        hour (int): Hour of mean diurnal profile to compute value for.
            Must be one of: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
            13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23].
            
    Returns:
        path_output (str): Output path for results from calc_diff.
        path_period1 (str): Output path for results from calc_func applied
            over the first period.
        path_period2 (str): Output path for results from calc_func applied
            over the second period.
    """
    
    # Assert that there are no errors in input arguments
    check_args(calc_func=calc_func, region=region, period_start=period1_start,
               period_end=period1_end, subset=subset, var_or_dvar=var_or_dvar,
               hour=hour)
    check_args(period_start=period2_start, period_end=period2_end, subset=subset)
    
    # Define path stems
    path_output = ("../data_processed/{}_diff_{}_{}_{}_{}_{}_{}_"
                   .format(calc_funcs_ver, region, period1_start.strftime("%b-%Y"),
                           period1_end.strftime("%b-%Y"), period2_start.strftime("%b-%Y"),
                           period2_end.strftime("%b-%Y"), subset)
                  )
    path_period1 = ("../data_processed/{}_{}_{}_{}_{}_"
                    .format(calc_funcs_ver, region, period1_start.strftime("%b-%Y"),
                            period1_end.strftime("%b-%Y"), subset)
                   )
    path_period2 = ("../data_processed/{}_{}_{}_{}_{}_"
                    .format(calc_funcs_ver, region, period2_start.strftime("%b-%Y"),
                            period2_end.strftime("%b-%Y"), subset)
                   )
    # Define path endings (exception for calc_glass_mean_climatology)
    if calc_func.__name__ == "calc_glass_mean_climatology":
        glass_data_source_period1 = glass_data_source_to_use(period1_start, period1_end)
        glass_data_source_period2 = glass_data_source_to_use(period2_start, period2_end)
        if glass_data_source_period1 == glass_data_source_period2:
            glass_data_source = glass_data_source_period1
        else:
            glass_data_source = "mixed"
        path_output += f"glass-mean_{glass_data_source}.nc"
        path_period1 += f"glass-mean_{glass_data_source_period1}.nc"
        path_period2 += f"glass-mean_{glass_data_source_period2}.nc"
        return path_output, path_period1, path_period2        
    if calc_func.__name__ == "calc_era5_mdp_climatology_given_var_or_dvar":
        path_ending = f"era5-mdp_{var_or_dvar}.nc"
    if calc_func.__name__ == "calc_era5_mdp_climatology_stats_given_var_or_dvar":
        path_ending = f"era5-mdp_{var_or_dvar}_stats.nc"        
    if calc_func.__name__ == "calc_era5_mdp_climatology_values_given_var_or_dvar_and_hour":
        path_ending = f"era5-mdp_{var_or_dvar}_{hour}.nc"        
    if calc_func.__name__ == "calc_era5_weibull_climatology":
        path_ending = "era5-weibull.nc"
    
    # Append path endings to stems and return
    path_period1 += path_ending
    path_period2 += path_ending
    path_output += path_ending
    return path_output, path_period1, path_period2

In [ ]:
## Low-level calc functions

def calc_glass_mean_climatology(region, period_start, period_end, subset,
                                var_or_dvar=None, hour=None):
    
    """
    Calculate mean leaf area index (MLAI) and mean fraction of absorbed
    photosynthetically active radiation (MFAPAR) climatology using GLASS data.
    
    Arguments:
        region (str): Region to perform calculation over.
            Must be one of ["ca", "sa", "wa"].
        period_start (str): Start of period to perform calculation over.
            Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
        period_end (str): End of period to perform calculation over.
            Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
        subset (str): Subset of period to perform calculation over.
            Must be one of ["all", "DJF", "MAM", "JJA", "SON",
            "Jan", "Feb", "Mar", "Apr", "May", "Jun",
            "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"].
                        
    Returns:
        {calc_funcs_ver}_{region}_{period_start}_{period_end}_{subset}_
        glass-mean_{glass_data_source}.nc:
            Output netcdf4 file in data_processed folder containing both MLAI and
            MFAPAR. {calc_funcs_ver} is the version of the calc_funcs script being
            used and {glass_data_source} is  automatically selected between
            ["avhrr", "modis"] based on the selected period.
    
    For each grid cell, calculate the mean glass climatology (MLAI or MFAPAR) over 
    the period from period_start to period_end (inclusive). The calculation uses 
    8-day satellite HDF data from the data_raw folder as input, then outputs 
    the result as a netcdf4 file into the data_processed folder. MODIS data
    is preferentially used where the given period is completely contained 
    within the time range of MODIS data. Otherwise, AVHRR data is used.
    """
    
    # Assert that there are no errors in input arguments, and select the
    # appropriate data source (AVHRR or MODIS) to use depending on period
    period_start = datetime.strptime(period_start, "%b-%Y")
    period_end = datetime.strptime(period_end, "%b-%Y")
    glass_data_source = glass_data_source_to_use(period_start, period_end)
    check_args(region=region, period_start=period_start, period_end=period_end,
               subset=subset)
    path_output = ("../data_processed/{}_{}_{}_{}_{}_glass-mean_{}.nc"
                   .format(calc_funcs_ver, region, period_start.strftime("%b-%Y"),
                           period_end.strftime("%b-%Y"), subset, glass_data_source)
                  )
    if Path(path_output).exists():
        print("Function was terminated because file already exists: " + path_output)
        return None

    # The two functions below are used with xarray's open_mfdataset for parallel
    # computing using dask. The region and times (period and subset) are
    # selected within separate functions and uses different logic as compared with
    # filtering in the ERA5 datasets. This is because each GLASS file contains
    # global data whereas the ERA5 datasets were downloaded for each local region.
    
    def filter_glass_files(file):
        # This function is used as a mask in conjunction with the default python
        # filter function later, in order to select out the raw data files within
        # the input period and climatological subset within the original function 
        # arguments (by using dates contained within each file name). This is done 
        # (as opposed to using open_mfdataset then filtering) for scalability reasons 
        # since we may need to persist the data in RAM to speed up certain computations.
        time = file[-12:-4]
        time = datetime.strptime(time, "%Y-%j")
        if ((time.month in subsets[subset]) &
            # We add an extra month to period_end here because period_end was
            # specified as a month, and conversion into a datetime object
            # defaults to the first (rather than last) day of that month
            (period_start <= time < period_end + relativedelta(months=1))
           ):
            return True
        else:
            return False
    
    def preprocess_glass(ds):
        # This function is used for the preprocess argument in open_mfdataset.
        # It uses the dates in each raw data file name to assign a time dimension
        # and coordinate for the corresponding dataset. This then forms the
        # dimension along which the files are combined into a single dataset
        # and rechunked. This function also selects out the input region within
        # the original function arguments before the files are concatenated using
        # open_mfdataset (this is again done for persist scalability).
        time = ds.encoding["source"][-12:-4]
        time = datetime.strptime(time, "%Y-%j")
        ds = (ds
              .expand_dims({"time": [time]})
              # Redundant measure just in case longitudes exceed 180 degrees
              .assign_coords({"x": (ds.x + 180) % 360 - 180})
              .sortby("x")
              .rename({"x": "longitude", "y": "latitude"})
              .drop_vars("spatial_ref")
              .squeeze("band", drop=True)
              )
        ds = ds.sel(longitude=slice(regions[region]["extent"][0],
                                    regions[region]["extent"][1]),
                    latitude=slice(regions[region]["extent"][3],
                                   regions[region]["extent"][2])
                   )
        return ds
    
    # The following code creates the mean climatology datasets for each GLASS
    # variable, by using the previous functions along with open_mfdataset.
    # An initally empty dataset is iteratively appended then merged so that
    # future scalability is possible in case one wishes to add more GLASS
    # variables to the vars_glass global python variable.
    datasets = []    
    for var in vars_glass:
        files_glass_all = glob(f"../data_raw/global_glass-{var}-{glass_data_source}" +
                               f"_8-day/global_glass-{var}-{glass_data_source}*")
        if len(files_glass_all) != number_of_glass_files[var][glass_data_source]:
            print(f"Warning: Expected {number_of_glass_files[var][glass_data_source]}" +
                  f" files in ../data_raw/global_glass-{var}-{glass_data_source}" +
                  f"_8-day/ but got {len(files_glass_all)}. This could be because the" +
                  " data_download.ipynb notebook was not run properly. Or it " +
                  "could be that the number of GLASS files on the server " +
                  "from which the data was downloaded has changed. Or it may " +
                  "be that the user has changed some files in this folder.")
        files_glass_filtered = list(filter(filter_glass_files, files_glass_all))
        files_glass_filtered.sort()
        # The if statement is to ensure an array full of NaNs is returned for MFAPAR
        # when the input period includes 1981 or 2021. At the time of writing, GLASS
        # FAPAR data is not available for these years.
        if (var == "fapar") & (
            (period_start < datetime.strptime(fapar_earliest, "%b-%Y")) |
            (period_end > datetime.strptime(fapar_latest, "%b-%Y"))
        ):
            # This line exploits the fact that the for loop runs in sequence and 
            # will have computed MLAI before it attempts to compute MFAPAR. 
            # Therefore an MLAI array with appropriate coordinates already exists 
            # in the datasets list and can be used to create an array of NaNs.
            ds_mean = (datasets[0]
                       .where(np.isnan(datasets[0]["mlai"]))
                       .rename({"mlai": "mfapar"})
                      )
            print("Warning: GLASS FAPAR data is not available for the years " +
                  f"{fapar_earliest[-4:]} and {fapar_latest[-4:]}. " +
                  "A data array with NaNs was returned for MFAPAR instead.")
        else:
            ds_mean = (xr.open_mfdataset(files_glass_filtered, engine = "rasterio",
                                         preprocess=preprocess_glass, parallel = True)
                       # Rechunking after open_mfdataset here is actually bad practice
                       # since it requires extra computation, but the chunks argument
                       # for open_mfdataset doesn't seem to work here for some reason.
                       .chunk(chunks = {"time": chunksize})
                       .persist()
                       .mean("time")
                       .rename({"band_data": "m" + var})
                      )
        datasets.append(ds_mean)
    ds_glass_mean = xr.merge(datasets)
    
    # Create output file in data_processed folder
    try:
        ds_glass_mean.to_netcdf(path_output)
        print("Created file: " + path_output)
    except:
        print("Failed to create file: " + path_output)

In [ ]:
%%time
calc_glass_mean_climatology("wa", "Jun-2001", "Aug-2006", "JJA")

In [ ]:
def calc_era5_weibull_climatology(region, period_start, period_end, subset,
                                  var_or_dvar=None, hour=None):

    """
    Calculate climatology of Weibull scale and shape parameters for wind speeds
    at 10 m and 100 m above surface using ERA5 data.
    
    Arguments:
        region (str): Region to perform calculation over.
            Must be one of ["ca", "sa", "wa"].
        period_start (str): Start of period to perform calculation over.
            Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
        period_end (str): End of period to perform calculation over.
            Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
        subset (str): Subset of period to perform calculation over.
            Must be one of ["all", "DJF", "MAM", "JJA", "SON",
            "Jan", "Feb", "Mar", "Apr", "May", "Jun",
            "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"].
                        
    Returns:
        {calc_funcs_ver}_{region}_{period_start}_{period_end}_{subset}_era5-weibull.nc:
            Output netcdf4 file in data_processed folder containing Weibull shape and
            scale parameters for wind speed at 10 m and 100 m above surface.
            {calc_funcs_ver} is the version of the calc_funcs script being used.
    
    For each grid cell, calculate the Weibull shape and scale parameters over 
    the period from period_start to period_end (inclusive). The calculation uses 
    hourly ERA5 netcdf4 data from the data_raw folder as input, then outputs 
    the result as a netcdf4 file into the data_processed folder.
    """
    
    # Assert that there are no errors in input arguments
    period_start = datetime.strptime(period_start, "%b-%Y")
    period_end = datetime.strptime(period_end, "%b-%Y")
    check_args(region=region, period_start=period_start, period_end=period_end,
               subset=subset)
    path_output = ("../data_processed/{}_{}_{}_{}_{}_era5-weibull.nc"
                   .format(calc_funcs_ver, region, period_start.strftime("%b-%Y"),
                           period_end.strftime("%b-%Y"), subset)
                  )
    if Path(path_output).exists():
        print("Function was terminated because file already exists: " + path_output)
        return None
    if period_start + relativedelta(years=5) > period_end:
        print("Warning: It is recommended to use at least 5 years of data " +
              "for estimating the Weibull parameters.")
    
    # The two functions below are used with xarray's open_mfdataset for parallel
    # computing using dask. Together they select out the relevant files to read
    # and persist in memory only the data which is necessary for the computation.
    
    def filter_era5_hour_files(file):
        # This function is used as a mask in conjunction with the default python
        # filter function later, in order to select out the raw data files with
        # years within the input period. The following preprocess function also
        # selects out the relevant years (as well as months) but by applying a
        # filter on the list of file names first we can avoid preprocessing a 
        # lot of files and hence save on memory.
        year = int(file[-7:-3])
        if period_start.year <= year <= period_end.year:
            return True
        else:
            return False
    
    def preprocess_era5_hour(ds):
        # This function is used for the preprocess argument in open_mfdataset.
        # It selects selects out only the wind speed data, and only for
        # the subset months. This is done for persist scalability.
        ds = (regrid_era5(ds)
              .sel(time = ds.time.dt.month.isin(subsets[subset]))
              .drop(["t2m", "msl", "slhf", "sshf"])
             )
        return ds
    
    # The following code opens the relevant hourly ERA5 files then uses the
    # u and v components of wind velocity to compute the magnitude (wind speed)
    files_era5_hour = glob(f"../data_raw/{region}_era5-slv-surface_hour/*.nc")
    files_era5_hour.sort()
    if len(files_era5_hour) != number_of_era5_hour_files:
        print(f"Warning: Expected {number_of_era5_hour_files} files in " +
              f"../data_raw/{region}_era5-slv-surface_hour/ but got " +
              f"{len(files_era5_hour)}. This could be because the " + 
              "data_download.ipynb notebook was not run properly. Or it " +
              "may be that the user has changed some files in this folder.")
    files_era5_hour_filtered = list(filter(filter_era5_hour_files, files_era5_hour))
    files_era5_hour_filtered.sort()
    ds_era5_hour = (xr.open_mfdataset(files_era5_hour_filtered, engine = "netcdf4",
                                     preprocess=preprocess_era5_hour, parallel = True)
                    # We add an extra month to period_end here because period_end was
                    # specified as a month, and conversion into a datetime object
                    # defaults to the first (rather than last) day of that month. The
                    # -1 hr is to avoid selecting first hour of the following month.
                    .sel(time = slice(period_start, period_end +
                                relativedelta(months=1, hours = -1)))
                    # Rechunking after open_mfdataset here is actually bad practice
                    # since it requires extra computation, but the chunks argument
                    # for open_mfdataset doesn't seem to work here for some reason.
                    .chunk(chunks = {"time": chunksize})
                    .persist()
                   )
    da_ws10 = magnitude(ds_era5_hour["u10"], ds_era5_hour["v10"])
    da_ws100 = magnitude(ds_era5_hour["u100"], ds_era5_hour["v100"])
    
    # Compute the mean and standard deviation of wind speed and from these the 
    # Weibull scale and shape parameters, then combine into a single dataset
    da_ws10_mean = xr.DataArray(da_ws10.mean("time"), name = "ws10_mean")
    da_ws10_std = xr.DataArray(da_ws10.std("time"), name = "ws10_std")
    da_ws10_c, da_ws10_k = weibull_params(da_ws10_mean, da_ws10_std)
    da_ws10_c.name, da_ws10_k.name = "c10", "k10"
    da_ws100_mean = xr.DataArray(da_ws100.mean("time"), name = "ws100_mean")
    da_ws100_std = xr.DataArray(da_ws100.std("time"), name = "ws100_std")
    da_ws100_c, da_ws100_k = weibull_params(da_ws100_mean, da_ws100_std)
    da_ws100_c.name, da_ws100_k.name = "c100", "k100"
    da_ws100_eroe = weibull_eroe(da_ws100_c, da_ws100_k, speed_eroe)
    da_ws100_gcf = gcf(da_ws100, speeds_common, powers_avg, power_nameplate)
    da_ws100_eroe.name, da_ws100_gcf.name = "eroe100", "gcf100"
    ds_era5_weibull = xr.merge([da_ws10_mean, da_ws10_std, da_ws10_c, da_ws10_k,
                                da_ws100_mean, da_ws100_std, da_ws100_c,
                                da_ws100_k, da_ws100_eroe, da_ws100_gcf])
    
    # Create output file in data_processed folder
    try:
        ds_era5_weibull.to_netcdf(path_output)
        print("Created file: " + path_output)
    except:
        print("Failed to create file: " + path_output)

In [ ]:
%%time
calc_era5_weibull_climatology("wa", "Dec-1984", "Feb-1995", "DJF")

In [ ]:
def calc_diff(calc_func, region, period1_start, period1_end,
              period2_start, period2_end, subset, var_or_dvar=None, hour=None):
    
    """
    Calculates the difference in results for two separate periods which have
    each been outputted by the same calculation function.
    
    Arguments:
        calc_func (function): Calculation function to compute difference in
            results from. Must be one of: ["calc_glass_mean_climatology",
            "calc_era5_mdp_climatology_given_var_or_dvar",
            "calc_era5_mdp_climatology_stats_given_var_or_dvar",
            "calc_era5_mdp_climatology_values_given_var_or_dvar_and_hour",
            "calc_era5_weibull_climatology"].
        region (str): Region to perform calculation over.
            Must be one of: ["ca", "sa", "wa"].
        period1_start (str): Start of first period to perform calculation over.
            Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
        period1_end (str): End of first period to perform calculation over.
            Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
        period2_start (str): Start of second period to perform calculation over.
            Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
        period2_end (str): End of second period to perform calculation over.
            Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
        subset (str): Subset of period to perform calculation over.
            Must be one of: ["all", "DJF", "MAM", "JJA", "SON",
            "Jan", "Feb", "Mar", "Apr", "May", "Jun",
            "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"].
        var_or_dvar (str): Variable or value of change in variable to perform
            calculation over. Must be one of: ['wv10', 'wv100', 'mslp', 't2',
            'slhf', 'sshf', 'viec', 'vipile', 'vike', 'tcclw', 'tcwv', 'nac',
            'dwv10', 'dwv100', 'dmslp', 'dt2', 'dslhf', 'dsshf', 'dviec',
            'dvipile', 'dvike', 'dtcclw', 'dtcwv', 'dnac'].
        hour (int): Hour of mean diurnal profile to compute value for.
            Must be one of: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
            13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23].
    
    Returns:
        {calc_funcs_ver}_diff_{region}_{period1_start}_{period1_end}_{period2_start}_
            {period2_end}_{subset}_glass-mean_{glass_data_source}.nc OR
        {calc_funcs_ver}_diff_{region}_{period1_start}_{period1_end}_{period2_start}_
            {period2_end}_{subset}_era5-mdp_{var_or_dvar}.nc OR
        {calc_funcs_ver}_diff_{region}_{period1_start}_{period1_end}_{period2_start}_
            {period2_end}_{subset}_era5-mdp_{var_or_dvar}_stats.nc OR
        {calc_funcs_ver}_diff_{region}_{period1_start}_{period1_end}_{period2_start}_
            {period2_end}_{subset}_era5-mdp_{var_or_dvar}_{hour}.nc OR
        {calc_funcs_ver}_{region}_{period1_start}_{period1_end}_{period2_start}_
            {period2_end}_{subset}_era5-weibull.nc:
                Output netcdf4 file in data_processed folder containing the difference
                in results, with name depending on calc_func being used.
                {calc_funcs_ver} is the version of the calc_funcs script being
                used and {glass_data_source} is automatically selected between
                ["avhrr", "modis"] based on the selected period.
    
    First runs calc_func for each of the given periods if this has not already
    been done. Then calculates the difference in results as period2 - period1.
    """
    
    # Assert that there are no errors in input arguments, and obtain the paths for
    # the calc_diff output as well as intermediate calc_func outputs from each period
    period1_start = datetime.strptime(period1_start, "%b-%Y")
    period1_end = datetime.strptime(period1_end, "%b-%Y")
    period2_start = datetime.strptime(period2_start, "%b-%Y")
    period2_end = datetime.strptime(period2_end, "%b-%Y")
    check_args(calc_func=calc_func, region=region, period_start=period1_start,
               period_end=period1_end, subset=subset, var_or_dvar=var_or_dvar,
               hour=hour)
    check_args(period_start=period2_start, period_end=period2_end, subset=subset)
    path_output, path_period1, path_period2 = get_paths_for_calc_diff(
        calc_func=calc_func, region=region, period1_start=period1_start,
        period1_end=period1_end, period2_start=period2_start, period2_end=period2_end,
        subset=subset, var_or_dvar=var_or_dvar, hour=hour)
    if Path(path_output).exists():
        print("Function was terminated because file already exists: " + path_output)
        return None
    
    # Create intermediate output files from each period if they don't already
    # exist, then read in these files as xarray datasets and compute difference
    if Path(path_period1).exists():
        print("Using existing file: " + path_period1)
    else:
        calc_func(region=region, period_start=period1_start.strftime("%b-%Y"),
                  period_end=period1_end.strftime("%b-%Y"), subset=subset,
                  var_or_dvar=var_or_dvar, hour=hour)
    if Path(path_period2).exists():
        print("Using existing file: " + path_period2)
    else:
        calc_func(region=region, period_start=period2_start.strftime("%b-%Y"),
                  period_end=period2_end.strftime("%b-%Y"), subset=subset,
                  var_or_dvar=var_or_dvar, hour=hour)
    ds_period1 = xr.open_dataset(path_period1, engine = "netcdf4")
    ds_period2 = xr.open_dataset(path_period2, engine = "netcdf4")
    ds_diff = ds_period2 - ds_period1
    
    # Create output file in data_processed folder
    try:
        ds_diff.to_netcdf(path_output)
        print("Created file: " + path_output)
    except:
        print("Failed to create file: " + path_output)
    

    # write up schema for wind velocity
    # if statement for hour_of_max and hour_of_min in stats to treat separately:
    # they need to be between -12 and +12
    # if required: add to the docstring the fact that wind vectors and hour_of_max/hour_of_min will be treated differently

In [ ]:
%%time
calc_diff(calc_glass_mean_climatology, "wa", "Jan-1991", "Dec-1991", "Jan-2011", "Dec-2011", "all")

In [ ]:
# calc + plot for years with greatest change in LAI + magnitude of change during those years (rolling averages?)

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
client.close()

In [ ]:
test = xr.open_dataset("../data_processed/v00_diff_wa_Jan-1984_Dec-1989_Jan-2004_Dec-2009_all_era5-weibull.nc")
test

In [ ]:
test["ws100_mean"].plot()

In [ ]:
test["ws100_std"].plot()

In [ ]:
test["c100"].plot()

In [ ]:
test["k100"].plot()

In [ ]:
test["eroe100"].plot()

In [ ]:
test["gcf100"].plot()

In [ ]:
test["eroe100"].where(test["eroe100"]==test["eroe100"].max(), drop = True).squeeze()

In [ ]:
def get_paths_for_calc_diff(calc_func, region, period1_start, period1_end, period2_start,
                            period2_end, subset, var_or_dvar=None, hour=None):
    
    """
    Obtain output path for calc_diff function, as well as paths for the calc_func
    outputs from each given period.
    
    Arguments:
        calc_func (function): Calculation function to compute difference in
            results from. Must be one of: ["calc_glass_mean_climatology",
            "calc_era5_mdp_climatology_given_var_or_dvar",
            "calc_era5_mdp_climatology_stats_given_var_or_dvar",
            "calc_era5_mdp_climatology_values_given_var_or_dvar_and_hour",
            "calc_era5_weibull_climatology"].
        region (str): Region to perform calculation over.
            Must be one of: ["ca", "sa", "wa"].
        period1_start (datetime.datetime): Start of first period to perform
            calculation over. Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
            Note that this is a datetime.dateime object as opposed to str.
        period1_end (datetime.datetime): End of first period to perform
            calculation over. Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
            Note that this is a datetime.dateime object as opposed to str.
        period2_start (datetime.datetime): Start of second period to perform
            calculation over. Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
            Note that this is a datetime.dateime object as opposed to str.
        period2_end (datetime.datetime): End of second period to perform
            calculation over. Must be of form "%b-%Y" eg. "Jul-1990".
            Must be between "Jan-1981" and "Dec-2021".
            Note that this is a datetime.dateime object as opposed to str.
        subset (str): Subset of period to perform calculation over.
            Must be one of: ["all", "DJF", "MAM", "JJA", "SON",
            "Jan", "Feb", "Mar", "Apr", "May", "Jun",
            "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"].
        var_or_dvar (str): Variable or value of change in variable to perform
            calculation over. Must be one of: ['wv10', 'wv100', 'mslp', 't2',
            'slhf', 'sshf', 'viec', 'vipile', 'vike', 'tcclw', 'tcwv', 'nac',
            'dwv10', 'dwv100', 'dmslp', 'dt2', 'dslhf', 'dsshf', 'dviec',
            'dvipile', 'dvike', 'dtcclw', 'dtcwv', 'dnac'].
        hour (int): Hour of mean diurnal profile to compute value for.
            Must be one of: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
            13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23].
            
    Returns:
        path_output (str): Output path for results from calc_diff.
        path_period1 (str): Output path for results from calc_func applied
            over the first period.
        path_period2 (str): Output path for results from calc_func applied
            over the second period.
    """
    
    if calc_func.__name__ == "calc_glass_mean_climatology":
        glass_data_source_period1 = glass_data_source_to_use(period1_start, period1_end)
        glass_data_source_period2 = glass_data_source_to_use(period2_start, period2_end)
        if glass_data_source_period1 == glass_data_source_period2:
            glass_data_source = glass_data_source_period1
        else:
            glass_data_source = "mixed"
        path_period1 = ("../data_processed/{}_{}_{}_{}_{}_glass-mean_{}.nc"
                        .format(calc_funcs_ver, region,
                                period1_start.strftime("%b-%Y"),
                                period1_end.strftime("%b-%Y"),
                                subset, glass_data_source_period1)
                       )
        path_period2 = ("../data_processed/{}_{}_{}_{}_{}_glass-mean_{}.nc"
                        .format(calc_funcs_ver, region,
                                period2_start.strftime("%b-%Y"),
                                period2_end.strftime("%b-%Y"),
                                subset, glass_data_source_period2)
                       )
        path_output = ("../data_processed/{}_diff_{}_{}_{}_{}_{}_{}_glass-mean_{}.nc"
                       .format(calc_funcs_ver, region,
                               period1_start.strftime("%b-%Y"),
                               period1_end.strftime("%b-%Y"),
                               period2_start.strftime("%b-%Y"),
                               period2_end.strftime("%b-%Y"),
                               subset, glass_data_source)
                      )
        
    if calc_func.__name__ == "calc_era5_mdp_climatology_given_var_or_dvar":
        path_period1 = ("../data_processed/{}_{}_{}_{}_{}_era5-mdp_{}.nc"
                        .format(calc_funcs_ver, region,
                                period1_start.strftime("%b-%Y"),
                                period1_end.strftime("%b-%Y"),
                                subset, var_or_dvar)
                       )
        path_period2 = ("../data_processed/{}_{}_{}_{}_{}_era5-mdp_{}.nc"
                        .format(calc_funcs_ver, region,
                                period2_start.strftime("%b-%Y"),
                                period2_end.strftime("%b-%Y"),
                                subset, var_or_dvar)
                       )
        path_output = ("../data_processed/{}_diff_{}_{}_{}_{}_{}_{}_era5-mdp_{}.nc"
                       .format(calc_funcs_ver, region,
                               period1_start.strftime("%b-%Y"),
                               period1_end.strftime("%b-%Y"),
                               period2_start.strftime("%b-%Y"),
                               period2_end.strftime("%b-%Y"),
                               subset, var_or_dvar)
                      )
        
    if calc_func.__name__ == "calc_era5_mdp_climatology_stats_given_var_or_dvar":
        path_period1 = ("../data_processed/{}_{}_{}_{}_{}_era5-mdp_{}_stats.nc"
                        .format(calc_funcs_ver, region,
                                period1_start.strftime("%b-%Y"),
                                period1_end.strftime("%b-%Y"),
                                subset, var_or_dvar)
                       )
        path_period2 = ("../data_processed/{}_{}_{}_{}_{}_era5-mdp_{}_stats.nc"
                        .format(calc_funcs_ver, region,
                                period2_start.strftime("%b-%Y"),
                                period2_end.strftime("%b-%Y"),
                                subset, var_or_dvar)
                       )
        path_output = ("../data_processed/{}_diff_{}_{}_{}_{}_{}_{}_era5-mdp_{}_stats.nc"
                       .format(calc_funcs_ver, region,
                               period1_start.strftime("%b-%Y"),
                               period1_end.strftime("%b-%Y"),
                               period2_start.strftime("%b-%Y"),
                               period2_end.strftime("%b-%Y"),
                               subset, var_or_dvar)
                      )
        
    if calc_func.__name__ == "calc_era5_mdp_climatology_values_given_var_or_dvar_and_hour":
        path_period1 = ("../data_processed/{}_{}_{}_{}_{}_era5-mdp_{}_{}.nc"
                        .format(calc_funcs_ver, region,
                                period1_start.strftime("%b-%Y"),
                                period1_end.strftime("%b-%Y"),
                                subset, var_or_dvar, hour)
                       )
        path_period2 = ("../data_processed/{}_{}_{}_{}_{}_era5-mdp_{}_{}.nc"
                        .format(calc_funcs_ver, region,
                                period2_start.strftime("%b-%Y"),
                                period2_end.strftime("%b-%Y"),
                                subset, var_or_dvar, hour)
                       )
        path_output = ("../data_processed/{}_diff_{}_{}_{}_{}_{}_{}_era5-mdp_{}_{}.nc"
                       .format(calc_funcs_ver, region,
                               period1_start.strftime("%b-%Y"),
                               period1_end.strftime("%b-%Y"),
                               period2_start.strftime("%b-%Y"),
                               period2_end.strftime("%b-%Y"),
                               subset, var_or_dvar, hour)
                      )
        
    if calc_func.__name__ == "calc_era5_weibull_climatology":
        path_period1 = ("../data_processed/{}_{}_{}_{}_{}_era5-weibull.nc"
                        .format(calc_funcs_ver, region,
                                period1_start.strftime("%b-%Y"),
                                period1_end.strftime("%b-%Y"),
                                subset)
                       )
        path_period2 = ("../data_processed/{}_{}_{}_{}_{}_era5-weibull.nc"
                        .format(calc_funcs_ver, region,
                                period2_start.strftime("%b-%Y"),
                                period2_end.strftime("%b-%Y"),
                                subset)
                       )
        path_output = ("../data_processed/{}_diff_{}_{}_{}_{}_{}_{}_era5-weibull.nc"
                       .format(calc_funcs_ver, region,
                               period1_start.strftime("%b-%Y"),
                               period1_end.strftime("%b-%Y"),
                               period2_start.strftime("%b-%Y"),
                               period2_end.strftime("%b-%Y"),
                               subset)
                      )
    
    return path_output, path_period1, path_period2